In [2]:
TABLA_CALENDARIO = 'LH_STORE_Oro.Calendario'
TABLA_ORDENES = 'LH_STORE_Oro.Ordenes'
TABLA_PRODUCTOS = 'LH_STORE_Oro.Productos'
TABLA_TERRITORIOS = 'LH_STORE_Oro.Territorios'

StatementMeta(, e8a5f16a-0bfa-47c6-805b-e2136035377b, 4, Finished, Available, Finished)

# Crear tabla Calendario


In [3]:
import pyspark.sql.functions as F

start_date = "2015-01-01"
end_date = "2015-12-31"

df = spark.createDataFrame([(start_date, end_date)], ["start", "end"])
df = df.select(F.sequence(F.to_date(F.lit(start_date)), F.to_date(F.lit(end_date))).alias("Fecha"))
df = df.select(F.explode("Fecha").alias("Fecha"))

# Agregar columnas de atributos del calendario
df = df \
    .withColumn("Año", F.year("Fecha")) \
    .withColumn("Trimestre", F.quarter("Fecha")) \
    .withColumn("NumMes", F.month("Fecha")) \
    .withColumn("Mes", F.date_format("Fecha", "MMMM")) \
    .withColumn("Día", F.dayofmonth("Fecha")) \
        

# Mostrar resultados
display(df.limit(10))

StatementMeta(, e8a5f16a-0bfa-47c6-805b-e2136035377b, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7991e34c-6b41-484a-b50e-8c1b8ba1fbc3)

In [4]:
from delta.tables import DeltaTable

if not spark.catalog.tableExists(TABLA_CALENDARIO):
    print('Creando la tabla')
    df.write.format("delta").saveAsTable(TABLA_CALENDARIO)
else:
    print('Haciendo MERGE')
    tabla_destino = DeltaTable.forName(spark, TABLA_CALENDARIO)
    tabla_destino.alias("target") \
        .merge(df.alias("source"), "source.Fecha = target.Fecha") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
           .execute()

        #.whenNotMatchedBySourceDelete() \
     


StatementMeta(, e8a5f16a-0bfa-47c6-805b-e2136035377b, 18, Finished, Available, Finished)

Haciendo MERGE


# Transformar Órdenes

In [1]:
df_Ordenes = spark.table("sales_salesorderdetail").toPandas()
# Mostrar resultados
display(df_Ordenes.head(10))

StatementMeta(, e8a5f16a-0bfa-47c6-805b-e2136035377b, 3, Finished, Available, Finished)

Cache file does not exists. resource=$ml, path=$/nfs4/tridenttokenlibrary/tokens/ml.token
failed to get ml token
Traceback (most recent call last):
  File "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/synapse/ml/fabric/token_utils.py", line 156, in _get_access_token_from_token_lib
    return PyTridentTokenLibrary.get_access_token_from_cache(resource)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/trident_token_library_wrapper.py", line 71, in get_access_token_from_cache
    raise IOError(err_msg)
OSError: Cache file does not exists. resource=$ml, path=$/nfs4/tridenttokenlibrary/tokens/ml.token


SynapseWidget(Synapse.DataFrame, 7b972f67-bfb3-49cf-9dce-6bde9e427126)

In [57]:
# Código generado por Data Wrangler para pandas DataFrame

def clean_data(df_Ordenes):
    # Cambiar el tipo de columna a int64para las columnas: 'SalesOrderID', 'OrderQty' y 2 otras columnas
    df_Ordenes = df_Ordenes.astype({'SalesOrderID': 'int64', 'OrderQty': 'int64', 'ProductID': 'int64', 'TerritoryID': 'int64'})
    # Eliminar columna: 'ModifiedDate'
    df_Ordenes = df_Ordenes.drop(columns=['ModifiedDate'])
    # Cambiar el tipo de columna a datetime64[ns]para las columnas: 'OrderDate', 'DueDate', 'ShipDate'
    df_Ordenes = df_Ordenes.astype({'OrderDate': 'datetime64[ns]', 'DueDate': 'datetime64[ns]', 'ShipDate': 'datetime64[ns]'})
    # Cambiar el tipo de columna a float64para las columnas: 'UnitPrice', 'UnitPriceDiscount'
    df_Ordenes = df_Ordenes.astype({'UnitPrice': 'float64', 'UnitPriceDiscount': 'float64'})
    return df_Ordenes

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 16, Finished, Available, Finished)

In [ ]:
df_Ordenes_clean = clean_data(df_Ordenes.copy())
display(df_Ordenes_clean)

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 17, Finished, Available, Finished)

/tmp/ipykernel_6497/4023367859.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_Ordenes = df_Ordenes.astype({'OrderDate': 'datetime64[ns]', 'DueDate': 'datetime64[ns]', 'ShipDate': 'datetime64[ns]'})
/tmp/ipykernel_6497/4023367859.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_Ordenes = df_Ordenes.astype({'OrderDate': 'datetime64[ns]', 'DueDate': 'datetime64[ns]', 'ShipDate': 'datetime64[ns]'})
/tmp/ipykernel_6497/4023367859.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_Ordenes = df_Ordenes.astype({'OrderDate': 'datetime64[ns]', 'DueDate': 'datetime64

SynapseWidget(Synapse.DataFrame, 84c60079-0b0a-4f8b-b435-8f190515a2f6)

In [58]:
df_OrdenesTransformadas = spark.createDataFrame(df_Ordenes_clean)
display(df_OrdenesTransformadas.limit(10))

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 56d778ce-4834-42b4-8c27-9b445b628961)

In [59]:
df_OrdenesTransformadas.write.mode("overwrite").format("delta").saveAsTable(TABLA_ORDENES)

   

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 19, Finished, Available, Finished)

# Transformar Productos

In [60]:
df_Productos = spark.table("LH_STORE_Plata.production_product")
display(df_Productos.limit(10))


StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3c215452-e44f-4ad0-93b4-9e7e504bc9a2)

In [61]:
# Código generado por Data Wrangler para PySpark DataFrame

from pyspark.sql import types as T

def clean_data(df_Productos):
    # Eliminar las columnas: 'ProductNumber', 'MakeFlag' y 17 otras columnas
    df_Productos = df_Productos.drop('ProductNumber', 'MakeFlag', 'FinishedGoodsFlag', 'SafetyStockLevel', 'ReorderPoint', 'StandardCost', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight', 'DaysToManufacture', 'ProductLine', 'Class', 'Style', 'ProductModelID', 'SellStartDate', 'SellEndDate', 'DiscontinuedDate', 'ModifiedDate')
    # Cambia el tipo de columna por int64 para la columna: 'ProductID'
    df_Productos = df_Productos.withColumn('ProductID', df_Productos['ProductID'].cast(T.LongType()))
    # Cambia el tipo de columna por float64 para la columna: 'ListPrice'
    df_Productos = df_Productos.withColumn('ListPrice', df_Productos['ListPrice'].cast(T.DoubleType()))
    return df_Productos

df_Productos_clean = clean_data(df_Productos)
display(df_Productos_clean)

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c71bc5f6-c855-4643-82c2-8a011bf9f89c)

In [62]:
df_Productos_clean.write.mode("overwrite").format("delta").saveAsTable(TABLA_PRODUCTOS)

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 22, Finished, Available, Finished)

# Transformar Territorios

In [63]:
df_Territorios = spark.table("LH_STORE_Plata.sales_salesterritory")
display(df_Territorios.limit(10))

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4e4c307c-6a78-46a9-932c-b307eabd6c83)

In [64]:
# Código generado por Data Wrangler para PySpark DataFrame

from pyspark.sql import types as T

def clean_data(df_Territorios):
    # Eliminar las columnas: 'CountryRegionCode', 'ModifiedDate'
    df_Territorios = df_Territorios.drop('CountryRegionCode', 'ModifiedDate')
    # Cambia el tipo de columna por int64 para la columna: 'TerritoryID'
    df_Territorios = df_Territorios.withColumn('TerritoryID', df_Territorios['TerritoryID'].cast(T.LongType()))
    # Cambiar el nombre de la columna 'Name' a 'Región'
    df_Territorios = df_Territorios.withColumnRenamed('Name', 'Región')
    # Cambiar el nombre de la columna 'Group' a 'Grupo'
    df_Territorios = df_Territorios.withColumnRenamed('Group', 'Grupo')
    return df_Territorios

df_Territorios_clean = clean_data(df_Territorios)
display(df_Territorios_clean)

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 212e3221-5e57-4e68-80bd-32d42b4057b4)

In [65]:
df_Territorios_clean.write.mode("overwrite").format("delta").saveAsTable(TABLA_TERRITORIOS)

StatementMeta(, 4fc1f56c-3a93-4631-8a9b-c2cea58fa100, 25, Finished, Available, Finished)